In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [13]:
def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        print('Fetch OK')
        return response.text
    else:
        print('Fetch ERROR')
        return None

---

### Handle price

code mới

In [ ]:
def specify_price_elements(block) -> bool:
    fare = block.find('div', class_='fare')      #  chỉ hiển thị 1 giá     
    
    if fare: # khong co gia khuyen mai
        return True
    
    return False

def parse_fare_small(block):

    fare_small = block.find('div', class_='fareSmall')
    
    original_price = fare_small.find('div', class_='small').get_text(strip=True).replace("đ", "").strip()
    
    if fare_small.find('div', class_='percent'):
        percent_discount = fare_small.find('div', class_='percent').get_text(strip=True)
    else:
        percent_discount = None
        
    return original_price, percent_discount

def parser_fare_sale(block):
    fare_sale = block.find('div', class_='fare-sale')
    original_price, percent_discount = parse_fare_small(block)
    
    if fare_sale and fare_sale.get_text(strip=True):
        sale_price = fare_sale.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()
        
    return original_price, sale_price, percent_discount

def parse_fare(block):
    fare = block.find('div', class_='fare')
    fare_price = fare.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()
    
    original_price = fare_price
    sale_price = None
    percent_discount = None
    
    return original_price ,sale_price, percent_discount  

def parser_price(block):
    if specify_price_elements(block):
        return parse_fare(block)
    else:
        return parser_fare_sale(block)

main

In [ ]:
def parse_price(block):
    fare_sale = block.find('div', class_='fare-sale')
    fare = block.find('div', class_='fare')
    small = block.find('div', class_='small')
    percent = block.find('div', class_='percent')

    # Discounted price
    if fare_sale:
        price_discount = fare_sale.get_text(strip=True).replace("Từ ", "").replace("đ", "").strip()
    elif fare:
        price_discount = fare.get_text(strip=True).replace("đ", "").strip()
    else:
        price_discount = None

    # Original price
    if small and small.get_text(strip=True):
        price_original = small.get_text(strip=True).replace("đ", "").strip()
    else:
        price_original = price_discount

    # Discount percent
    discount_percent = percent.get_text(strip=True) if percent else None

    return [price_original, price_discount, discount_percent]

---

### Parser bus infor

In [15]:
def parse_bus_info(container):
    bus_name = container.find('div', class_='bus-name').text.strip() if container.find('div', class_='bus-name') else None
    bus_rating = container.find('div', class_='bus-rating').find('span').text.strip() if container.find('div', class_='bus-rating') and container.find('div', class_='bus-rating').find('span') else None
    seat_type = container.find('div', class_='seat-type').get_text(strip=True) if container.find('div', class_='seat-type') else None

    from_to_content = container.find('div', class_="from-to-content")
    if from_to_content:
        to_content = from_to_content.find('div', class_='content to')
        from_content = from_to_content.find('div', class_='content from')
        duration = from_to_content.find('div', class_="duration").get_text(strip=True) if from_to_content.find('div', class_="duration") else None

        # Arrival info
        date_arrival = None
        to_hour = None
        to_place = None
        if to_content:
            span = to_content.find('span', class_="text-date-arrival-time")
            date_arrival = span.get_text(strip=True) if span else None
            content_to_info = to_content.find('div', class_='content-to-info')
            if content_to_info:
                to_hour = content_to_info.find('div',class_='hour' ).get_text(strip=True) if content_to_info.find('div',class_='hour' ) else None
                to_place = content_to_info.find('div',class_='place' ).get_text(strip=True) if content_to_info.find('div',class_='place' ) else None

        # Departure info
        from_hour = from_content.find('div',class_='hour' ).get_text(strip=True) if from_content and from_content.find('div',class_='hour' ) else None
        from_place = from_content.find('div',class_='place' ).get_text(strip=True) if from_content and from_content.find('div',class_='place' ) else None
    else:
        duration = None
        date_arrival = None
        to_hour = None
        to_place = None
        from_hour = None
        from_place = None

    price_original, price_discount, discount_percent = parse_price(container)

    notification = container.find('div', class_='link')
    notification = notification.get_text(strip=True) if notification else None

    return [
        bus_name, bus_rating, seat_type,
        from_hour, from_place, duration,
        to_hour, to_place, date_arrival,
        price_original, price_discount, discount_percent, notification
    ]

#### Convert from List to DataFrame

In [16]:
def extract_all_bus_info(soup) -> list:
    containers = soup.find_all("div", class_="container")
    bus_info_list = [parse_bus_info(container) for container in containers]
    return bus_info_list

---

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time, random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


### Click button 'Xem thêm chuyến'

In [17]:
def click_load_more(driver):
    while True:
        try:
            load_more_span = driver.find_element(By.XPATH, "//span[text()='Xem thêm chuyến']")
            load_more_button = load_more_span.find_element(By.XPATH, "./ancestor::button")
            driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
            load_more_button.click()
            time.sleep(random.uniform(2, 3))
        except NoSuchElementException:
            break
        except ElementClickInterceptedException:
            time.sleep(2)

# FLOW OFFICIAL

In [18]:



# driver = webdriver.Chrome()
# driver.get("https://vexere.com/vi-VN/ve-xe-khach-tu-sai-gon-di-nha-trang-khanh-hoa-129t23591.html?date=11-10-2025&v=6")
# click_load_more(driver)

# # After all trips are loaded, get page source
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')

# bus_info_list = []
# bus_seen = set()
# containers = soup.find_all("div", class_="container")
# for container in containers:
#     bus_name = container.find('div', class_='bus-name').text.strip() if container.find('div', class_='bus-name') else None
#     bus_rating = container.find('div', class_='bus-rating').find('span').text.strip() if container.find('div', class_='bus-rating') and container.find('div', class_='bus-rating').find('span') else None
#     key = (bus_name, bus_rating)
#     if key in bus_seen:
#         # Already have rating for this bus, skip opening rating
#         pass
#     else:
#         # If rating not visible, open rating (not implemented, placeholder)
#         # TODO: open rating if not visible
#         bus_seen.add(key)
#     bus_info_list.append(parse_bus_info(container))

# df_bus_info = pd.DataFrame(bus_info_list, columns=[
#     'bus_name', 'bus_rating', 'seat_type',
#     'from_hour', 'from_place', 'duration',
#     'to_hour', 'to_place', 'date_arrival',
#     'price_original', 'price_discount', 'discount_percent', 'notification'
# ])
# df_bus_info.to_csv("bus_info.csv", index=False)
# driver.quit()
# df_bus_info

---

## Crawl rating data

In [19]:
def extract_rating_from_container(container):
    try:
        ratings = []
        rate_divs = container.find_all('div', class_='rate-title')
        for rate_div in rate_divs:
            rate_ps = rate_div.find_all('p')
            if len(rate_ps) >= 2:
                rate_title = rate_ps[0].get_text(strip=True)
                rate_point = rate_ps[1].get_text(strip=True)
                ratings.append((rate_title, rate_point))
        if ratings:
            return ratings
        else:
            return [(None, None)]
    except Exception:
        return [(None, None)]

# -- Main --

In [ ]:

driver = webdriver.Chrome()
driver.get("https://vexere.com/vi-VN/ve-xe-khach-tu-sai-gon-di-nha-trang-khanh-hoa-129t23591.html?date=11-10-2025&v=6")

# click_load_more(driver)

# Open all ratings after all trips are loaded
try:
    stars = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".bus-rating-button .anticon-star"))
    )
    for star in stars:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
        time.sleep(1)  # tạm nghỉ nhẹ để trang ổn định
        ActionChains(driver).move_to_element(star).click().perform()
        time.sleep(2)
except Exception as e:
    print("⚠️ Error clicking star icons:", e)

time.sleep(3)
soup = BeautifulSoup(driver.page_source, "html.parser")

all_trips_info = []
columns_trips_info = [
    'bus_company', 'bus_star', 'bus_type',
    'departure_time', 'pick_up_point', 'duration_h_m',
    'arrival_time', 'drop_off_point', 'arrival_date',
    'price_original', 'price_discount', 'discount_percent', 'notification',
]

containers = soup.find_all("div", class_="container")
for container in containers:
    bus_info = parse_bus_info(container)
    ratings = extract_rating_from_container(container)
    
    # --- Xử lý df_ratings ---
    df_ratings = pd.DataFrame(ratings, columns=['rate_title', 'rate_point']).T
    header_row = df_ratings.iloc[0] 
    df_ratings.columns = header_row
    
    
    df_ratings = df_ratings[1:]
    
    df_ratings = df_ratings.reset_index(drop=True)
    
    # --- Xử lý df_bus_info ---
    df_bus_info = pd.DataFrame([bus_info], columns=columns_trips_info)
    
   
    df_bus_info = df_bus_info.reset_index(drop=True)

    # --- Nối ngang ---
    
    trip_info = pd.concat([df_bus_info, df_ratings], axis=1)
    all_trips_info.append(trip_info)

all_trips_info = pd.concat(all_trips_info, ignore_index=True)
all_trips_info.to_csv("../data/bus_info_with_rating.csv", index=False)
driver.quit()
all_trips_info

,bus_company,bus_star,bus_type,departure_time,pick_up_point,duration_h_m,arrival_time,drop_off_point,arrival_date,price_original,...,discount_percent,notification,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ,None
0,Đà Lạt ơi,4.8 (3425),Limousine 24 Phòng ĐÔI,23:45,• Trạm Quận 1,5h45m,05:30,• Trạm Nha Trang,(12/10),450.000,...,-29%,Lộ trình: Hàng Xanh - Quận 1 - Cao tốc đến Cam...,4.8,4.8,4.8,4.8,4.8,4.8,4.8,NaN
1,Bình Minh Tải,4.7 (3650),Limousine 22 Phòng Đơn,22:30,• Văn Phòng Quận 1,7h5m,05:35,• Văn phòng Nha Trang,(12/10),350.000,...,-14%,Hướng đi: Cao tốc (Long Thành - Dầu Giây - Pha...,4.6,4.6,4.7,4.6,4.6,4.6,4.8,NaN
2,Huỳnh Gia,4.7 (8534),Giường nằm 38 chỗ (WC),22:30,• Văn Phòng Phạm Ngũ Lão,6h30m,05:00,• Văn Phòng Nha Trang,(12/10),280.000,...,-11%,Lộ trình: Cao tốc (Long Thành - Dầu Giây - Pha...,4.8,4.7,4.7,4.7,4.6,4.6,4.7,NaN
3,Khanh Phong,4.7 (16845),Limousine 20 giường phòng (WC),05:15,• Văn Phòng Phạm Ngũ Lão - Quận 1.,6h10m,11:25,• Văn Phòng Nha Trang (KS Mường Thanh),None,480.000,...,-17%,Quý khách lưu ý,4.8,4.7,4.7,4.7,4.6,4.6,4.8,NaN
4,An Anh Limousine,4.8 (8322),Limousine 34 Phòng Đơn,23:30,• Văn Phòng Quận 5,6h30m,06:00,• Văn phòng Nha Trang,(12/10),299.000,...,-33%,Tiện ích miễn phí,4.8,4.7,4.7,4.7,4.7,4.7,4.6,NaN
5,Nam Hải Limousine,4.7 (3463),Limousine 34 giường,22:20,• Văn Phòng Phạm Ngũ Lão,8h,06:20,• Văn Phòng Nha Trang,(12/10),300.000,...,-10%,Lưu ý Đón/Trả tại TP.HCM,4.8,4.7,4.7,4.7,4.7,4.7,4.8,NaN
6,Nhật Dương - Bình Minh Bus,4.9 (5826),Limousine 22 Phòng Đôi Luxury (WC),23:30,• Văn phòng Nguyễn Cư Trinh Quận 1,6h36m,06:06,• Vp Thích Quảng Đức Nha Trang,(12/10),349.000,...,-13%,NX xuất HĐ VAT - Hướng đi: Cao tốc,4.9,4.9,4.9,4.9,4.9,4.8,4.8,NaN
7,Bus365,4.7 (165),Limousine 24 phòng đôi,22:00,• Bến Xe Miền Đông Mới,6h,04:00,• Văn Phòng Nha Trang,(12/10),400.000,...,-28%,Vé Metro miễn phí,4.8,4.6,4.7,4.8,4.7,4.7,4.4,NaN
8,Trọng Thủy Limousine,4.7 (1344),Limousine 24 phòng Đôi,21:00,• Ngã 4 An Sương,7h,04:00,• Văn phòng Nha Trang,(12/10),470.000,...,-11%,Kích thước giường cuối,4.7,4.6,4.7,4.7,4.7,4.6,4.4,NaN
9,Liên Hưng,4 (12922),Limousine 21 phòng đơn (WC),22:15,• Bến xe Miền Tây (Quầy 24),8h45m,07:00,• Văn phòng Nha Trang,(12/10),500.000,...,-40%,None,4.3,4.2,4.3,4.1,4.0,3.9,4.2,NaN


In [21]:
all_trips_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   bus_company            20 non-null     object
 1   bus_star               20 non-null     object
 2   bus_type               20 non-null     object
 3   departure_time         20 non-null     object
 4   pick_up_point          20 non-null     object
 5   duration_h_m           20 non-null     object
 6   arrival_time           20 non-null     object
 7   drop_off_point         20 non-null     object
 8   arrival_date           15 non-null     object
 9   price_original         20 non-null     object
 10  price_discount         20 non-null     object
 11  discount_percent       20 non-null     object
 12  notification           18 non-null     object
 13  An toàn                19 non-null     object
 14  Thông tin chính xác    19 non-null     object
 15  Thông tin đầy đủ       19